In [1]:
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import os
import random
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
browser = webdriver.Chrome(executable_path=r'C:\Tools\chromedriver_win32\chromedriver.exe', options=option)
browser.implicitly_wait(10)

pageNumber = 2
baseUrl = 'https://gall.dcinside.com/board/lists/?id=earphone&page=' + str(pageNumber)  # '이어폰,헤드폰 갤러리'

browser.get(baseUrl)

<ipython-input-1-31396e25dae2>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r'C:\Tools\chromedriver_win32\chromedriver.exe', options=option)


In [2]:
def get_review_titles(tr_num):
    review_titles = []
    for i in range(3, tr_num):
        #print('title i=', i)
        try:
            review_title = browser.find_elements_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(' + str(i) + ') > td.gall_tit.ub-word > a')
        except Exception as e:
            print(e.message, 'i=', i)
            review_title = browser.find_elements_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(' + str(i) + ') > td.gall_tit.ub-word > a:nth-child(1)')
            
        review_titles.append(review_title[0].text)
    return review_titles
        
#titles = get_review_titles()
# for title in titles:
#     print(title)

In [3]:
def get_review_dates(tr_num):
    review_dates = []
    for i in range(3, tr_num):
        review_date = browser.find_elements_by_xpath('//*[@id="container"]/section[1]/article[2]/div[2]/table/tbody/tr[' + str(i) + ']/td[4]')        
        review_dates.append(review_date[0].get_attribute("title"))
    return review_dates

#dates = get_review_dates()
# for da in dates:
#     print(da)

In [39]:
import threading
def hello(): 
    time.sleep(3)
    webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform()


In [40]:
def get_review_contents(tr_num):
    review_contents = []
    for i in range(3, tr_num):
        btn_review_content = None
        if i == 3:
            try:
                btn_review_content = browser.find_element_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(3) > td.gall_tit.ub-word > a')
            except Exception as e:
                print(e.message, "i=", i)
                btn_review_content = browser.find_element_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(3) > td.gall_tit.ub-word > a:nth-child(1)')
        else:
            btn_review_content = browser.find_element_by_css_selector('#bottom_listwrap > section.left_content > article > div.gall_listwrap.list > table > tbody > tr:nth-child(' + str(i) + ') > td.gall_tit.ub-word > a:nth-child(1)')
        
        th = threading.Thread(target = hello, args = ())
        th.start() 
        time.sleep(1)
        btn_review_content.click()
        th.join()        
        
        #webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform() # not to load page infinitely
        #WebDriverWait(btn_review_content, timeout=3).until(lambda btn: btn.click())
        el = WebDriverWait(browser, timeout=3).until(lambda b: b.find_element_by_xpath("//*[@class='write_div']"))

        review_contents.append(el.text)
        #time.sleep(random.uniform(1,3))
        #webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform() # not to load page infinitely
        #time.sleep(2)        
    return review_contents

#contents = get_review_contents()
# for cont in contents:
#     print(cont)

In [41]:
def get_review():
    bs = BeautifulSoup(browser.page_source, 'html.parser')
    trs = bs.select('tbody tr')
    tr_num = len(trs) + 1
    print('tr_num=', tr_num)
    review_titles = get_review_titles(tr_num)
    review_dates = get_review_dates(tr_num)
    review_contents = get_review_contents(tr_num)

    df = pd.DataFrame(columns=['title','date','content'])
    for i in range(len(review_titles)):
        df = df.append({'title':review_titles[i],
                 'date':review_dates[i],
                 'content':review_contents[i]}, ignore_index=True)
    return(df)

In [42]:
save_file_path = 'C:/Users/USER/Week 7 실습코드'
os.chdir(save_file_path)

all_review = pd.DataFrame(columns=["title","date",'content'])
for i in range(331, 1000):
    pageNumber = i
    baseUrl = 'https://gall.dcinside.com/board/lists/?id=earphone&page=' + str(pageNumber)  # '이어폰,헤드폰 갤러리'   
    
    th = threading.Thread(target = hello, args = ())
    th.start()
    time.sleep(1)
    browser.get(baseUrl)
    th.join()
    
    #time.sleep(random.uniform(1,3))
    #webdriver.ActionChains(browser).send_keys(Keys.ESCAPE).perform() # not to load page infinitely
    #time.sleep(3)
    
    review = get_review()
    all_review = pd.concat([all_review, review], ignore_index=True)
    if i % 10 == 0:
        csv_filename = 'dcinside_earphone_' + str(i) + '.csv'
        all_review.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        all_review = pd.DataFrame()
        all_review = pd.DataFrame(columns=["title","date",'content'])
        
    
print("Crawling Finished")


#browser.close()

tr_num= 53


<ipython-input-2-bc561a84ee04>:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review_title = browser.find_elements_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(' + str(i) + ') > td.gall_tit.ub-word > a')
<ipython-input-3-d828271cd328>:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review_date = browser.find_elements_by_xpath('//*[@id="container"]/section[1]/article[2]/div[2]/table/tbody/tr[' + str(i) + ']/td[4]')
<ipython-input-40-1e7efebfda68>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn_review_content = browser.find_element_by_css_selector('#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(3) > td.gall_tit.ub-word > a')
<ipython-input-40-1e7efebfda68>:22: Deprecation

tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 22
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53
tr_num= 53

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E70C43+2493507]
	Ordinal0 [0x00E0A4B1+2073777]
	Ordinal0 [0x00D12608+1058312]
	Ordinal0 [0x00D04E8B+1003147]
	Ordinal0 [0x00D03EDD+999133]
	Ordinal0 [0x00D042A7+1000103]
	Ordinal0 [0x00D0DD6F+1039727]
	Ordinal0 [0x00D18039+1081401]
	Ordinal0 [0x00D1A6CB+1091275]
	Ordinal0 [0x00D04585+1000837]
	Ordinal0 [0x00D17EF9+1081081]
	Ordinal0 [0x00D65310+1397520]
	Ordinal0 [0x00D5580B+1333259]
	Ordinal0 [0x00D32314+1188628]
	Ordinal0 [0x00D3316F+1192303]
	GetHandleVerifier [0x00FF7BF6+1548950]
	GetHandleVerifier [0x010A461C+2256060]
	GetHandleVerifier [0x00EFC13B+518107]
	GetHandleVerifier [0x00EFB1E0+514176]
	Ordinal0 [0x00E0F53D+2094397]
	Ordinal0 [0x00E13418+2110488]
	Ordinal0 [0x00E13552+2110802]
	Ordinal0 [0x00E1CE81+2150017]
	BaseThreadInitThunk [0x758DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x773D7A9E+286]
	RtlGetAppContain

Exception in thread Thread-5261:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-39-4cdead2ecda4>", line 4, in hello
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\action_chains.py", line 75, in perform
    self.w3c_actions.perform()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\actions\action_builder.py", line 77, in perform
    self.driver.execute(Command.W3C_ACTIONS, enc)
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 418, in execute
    self.error_handler.check_response(response)
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 243, in check_response
    raise exception_class(message, screen, stacktrace)
